# Install

In [ ]:
##If you're curious about the GPU you've been assigned
!nvidia-smi

In [ ]:
####Only run this the first time!###
#Subsequently, run the next block with pull command
!git clone https://github.com/charlesabouhaidar/COMP473-Project.git
import os
os.chdir('COMP473-Project')
os.chdir('CycleGAN_simple')
!pip install -r requirements.txt

In [ ]:
##This for sure doesn't work, you need to start the repo from the branch you're working on in the first place
#!git checkout <branch_name>

In [ ]:
## Update colab to reflect changes made to repo, Honestly not sure this works though.
!git pull

# Datasets
Download one of the official datasets below (ordered by size, smallest to largest):
<ul>
    <li>grumpifycat</li>
    <li>facades</li>
    <li>apple2orange</li>
    <li>horse2zebra</li>
    <li>summer2winter_yosemite</li>
    <li>cezanne2photo</li>
    <li>ukiyoe2photo</li>
    <li>monet2photo</li>
    <li>vangogh2photo</li>
    <li>iphone2dslr_flower</li>
    <li>maps</li>
   </ul>

In [ ]:
!bash ./datasets/download_cyclegan_datasets.sh vangogh2photo

# Training

Note that all training done on colab will not be saved. After running the training script below make sure to save the files locally on your computer using the appropriate cell-blocks in "Save Checkpoints Locally", to keep the results.
<ul>
    <li>--dataroot: the directory for the data you downloaded above</li>
    <li>--name: the name you'd like to give your experiment</li>
    <li>--n_epochs: The number of epochs to train without weight decay</li>
    <li>--n_epochs_decay: The number of epochs to train with weight decay, these occur after n_epochs</li>
    <li>There are more options for you to explore in options/base_options.py and option/train_options.py</li>
   </ul>
For experiment names I think it'd be helpful if we adopted the format dataset_name for example vangogh2photo_smalldiscriminator.

In [ ]:
!python train.py --dataroot ./datasets/summer2winter_yosemite --name <experiment_name> --n_epochs=40 --n_epochs_decay=40

### Continue Training
Make sure to upload the checkpoint files before running the next block

In [ ]:
!python train.py --dataroot ./datasets/vangogh2photo --name st!python train.py --dataroot ./datasets/summer2winter_yosemite --name summer2winter_medium_nets --model cycle_gan --display_id 0 --n_epochs=60 --n_epochs_decay=60 --netG="resnet_6blocks" --continue_train --epoch_count=121

### Save Checkpoints Locally
Google Colab is definitely a bit annoying for this part. We have to manually save the checkpoints and then push them to the repo for the next time, or upload them directly to colab for continued training later.

In [ ]:
##saving images from checkpoints takes too long and risks a time-out. Getting just the latest models seems best
from google.colab import files
files.download("model_checkpoints/vangogh2photo_cycleweight1/latest_net_D_A.pth")
files.download("model_checkpoints/vangogh2photo_cycleweight1/latest_net_D_B.pth")
files.download("model_checkpoints/vangogh2photo_cycleweight1/latest_net_G_A.pth")
files.download("model_checkpoints/vangogh2photo_cycleweight1/latest_net_G_B.pth")
files.download("model_checkpoints/vangogh2photo_cycleweight1/loss_log.txt")
files.download("model_checkpoints/vangogh2photo_cycleweight1/train_opt.txt")

In [ ]:
##this saves all the other model checkpoints but no images
from google.colab import files
!rm -r model_checkpoints/vangogh2photo_cycleweight1/web
!zip -r model_checkpoints/vangogh2photo_cycleweight1.zip model_checkpoints/vangogh2photo_cycleweight1
files.download("model_checkpoints/vangogh2photo_cycleweight1.zip")

# Testing / Inference
Same deal as training. To save the test results run the "Save Results Locally" right after.

In [ ]:
!python test.py --dataroot datasets/summer2winter_yosemite --name <experiment_name> --no_dropout --model cycle_gan

### Save Results Locally
Just change the <experiment_name\> to match the same name you used while training

In [ ]:
!zip -r results/<experiment_name>.zip results/<experiment_name>/test_latest
files.download("results/<experiment_name>_results.zip")

# Experiments

### Linear Weight Increase on Cycle Loss (Short name: cycleweight)
Defining feature of experiment:
<p>Added a linear increase in the weight of the cycle consistency loss of main objective function. Where $k$ is the epoch number lambda was set accordingly. Extra hyperparameters for this experiment are $\lambda_1$ and $c$.$$\lambda_1 = 1.0, \, \lambda_k = \min(1.0 + c*k, 20)$$</p>

<hr style="height:5px">
<h5> vangogh2photo_cycleweight1 Training Parameters</h5>
<ul>
<li>Dataset: vangogh2photo (6287 images)</li>
    <li>Number of Epochs: 80</li>
<li>Learning Rate: 0.0002</li>
<li>batch size: 1</li>
<li>image size: 128 x 128</li>
<li>discriminators: 2 layer PatchGAN with 70 x 70 window</li>
<li>generators: 6 block Residual Network </li>
<li>pool size: 50</li>
     <li>$\lambda_1$: 1</li>
    <li>$c$: 0.1</li>
</ul>
<i>***Training halted due to poor results at 80 epoch mark. Notice red artefacts in our result images below. We believe this is due to using too small a discriminator.</i>

In [ ]:
!python train.py --dataroot ./datasets/vangogh2photo --name vangogh2photo_cycleweight1 --n_epochs=20 --n_epochs_decay=0 --cycle_weight=0.9 --cycle_weigth_inc=0.1 --save_latest_freq=125740 --save_epoch_freq=20 --print_freq=6287

<h5>vangogh2photo_cycleweight1 Training Losses</h5>

In [11]:
loss_to_graph("vangogh2photo_cycleweight1", "report_resources/logs/vangogh2photo_cycleweight1_loss_log.txt")

ValueError: could not convert string to float: '======'

<h5>Inference Images</h5>
 <img src="report_resources/images/vangogh2photo_cycleweight1.png" style="width:1000px;height:500px"/>

<hr style="height:5px">
<h5> horse2zebra_cycleweight2 Training Parameters</h5>
<ul>
<li>Dataset: horse2zebra (1334 images)</li>
    <li>Number of Epochs: 200, 100 without weight decay followed by 100 with weight decay to 0</li>
<li>Learning Rate: 0.0002</li>
<li>batch size: 1</li>
<li>image size: 128 x 128</li>
<li>discriminators: 3 layer PatchGAN with 70 x 70 window</li>
<li>generators: 6 block Residual Network </li>
<li>pool size: 50</li>
     <li>$\lambda_1$: 1</li>
    <li>$c$: 0.1</li>
</ul>

In [ ]:
!python train.py --dataroot ./datasets/horse2zebra --name horse2zebra_cycleweight2 --n_epochs=20 --n_epochs_decay=0 --cycle_weight=0.9 --cycle_weigth_inc=0.1 --save_latest_freq=125740 --save_epoch_freq=20 --print_freq=6287

<h5> horse2zebra_cycleweight2 Training Losses</h5>

In [10]:
loss_to_graph("horse2zebra_cycleweight2", "report_resources/logs/horse2zebra_cycleweight2_loss_log.txt")

ValueError: could not convert string to float: '======'

<h5>Inference Images</h5>
Results from the paper are on the left with our results on the same images on the right. You can see that ours kept from changing the background colours as much and overall wasn't as zealous with adding strips to the images.<br>

 <img src="report_resources/images/horse2zebra_cycleweight2_1.png" style="width:800px;height:200px"/>
 <img src="report_resources/images/horse2zebra_cycleweight2_2.png" style="width:800px;height:200px"/>
 <img src="report_resources/images/horse2zebra_cycleweight2_3.png" style="width:800px;height:200px"/>
 <img src="report_resources/images/horse2zebra_cycleweight2_4.png" style="width:800px;height:200px"/>

<hr style="height:5px">

## Utility Functions

In [ ]:
import matplotlib.pyplot as plt

In [7]:
def loss_to_graph(experiment_name, path_to_loss_file):
  counter = 1
  G_A_Loss = []
  G_B_Loss = []
  D_A_Loss = []
  D_B_Loss = []
  Cycle_A = []
  Cycle_B = []
  idt_A = []
  idt_B = []

#line looks likes
#(epoch: 9, iters: 2891, time: 0.371, data: 0.001) D_A: 0.123 G_A: 0.724 cycle_A: 1.047 idt_A: 0.401 D_B: 0.145 G_B: 0.715 cycle_B: 0.866 idt_B: 0.478
  with open(path_to_loss_file) as fp:
      for i, line in enumerate(fp):
          if(i == 0):continue #skip first line of stats
          position = line.find("G_A: ")
          G_A_Loss.append(line[position+5 : position +11]) 

          position = line.find("G_B: ")
          G_B_Loss.append(line[position+5 : position +11]) 

          position = line.find("D_A: ")
          D_A_Loss.append(line[position+5 : position +11]) 

          position = line.find("D_B: ")
          D_B_Loss.append(line[position+5 : position +11]) 
        
          position = line.find("cycle_A: ")
          Cycle_A.append(line[position+9 : position +15]) 

          position = line.find("cycle_B: ")
          Cycle_B.append(line[position+9 : position +15])

          position = line.find("idt_A: ")
          idt_A.append(line[position+7 : position +13]) 
        
          position = line.find("idt_B: ")
          idt_B.append(line[position+7 : position +13]) 

          counter = counter +1

#Generator losses
  G_A = list(map(float, G_A_Loss))
  G_B = list(map(float, G_B_Loss))
# Discriminator losses  
  D_A = list(map(float, D_A_Loss))
  D_B = list(map(float, D_B_Loss))
# Cycle consistency losses
  C_A = list(map(float, Cycle_A)) 
  C_B = list(map(float, Cycle_B)) 
# Identity losses
  ID_A = list(map(float, idt_A))
  ID_B = list(map(float, idt_B))
  experiment_name = "Relative Discriminator" ## LINE TO BE DELETED, WILL BE RECIEVIED AT THE TOP OF THE FUNCTION

#x axis
  x = list(range(1, len(G_A_Loss)+1))

#y axis
  fig = plt.figure(figsize=(16,20))
  fig.suptitle(experiment_name)
  ax1 = fig.add_subplot(311) ## generator plot
  ax2 = fig.add_subplot(312) ## discriminator plot
  ax3 = fig.add_subplot(313) ## Cycle consistency plot

#Graph 1
  ax1.set_title('Generators')
  ax1.set_xlabel('epochs')
  ax1.set_ylabel('error loss')
  ax1.plot(x,G_A, color="blue", label="Generator A")
  ax1.plot(x,G_B, color="red", label="Generator B")
  ax1.legend()
#Graph 2
  ax2.set_title('Discriminators')
  ax2.set_xlabel('epochs')
  ax2.set_ylabel('error loss')
  ax2.plot(x,D_A, color="blue", label="Discriminator A")
  ax2.plot(x,D_B, color="red", label="Discriminator B")
  ax2.legend()
#Graph 3
  ax3.set_title('Cycle losses')
  ax3.set_xlabel('epochs')
  ax3.set_ylabel('error loss')
  ax3.plot(x,C_A, color="blue", label="Cycle A")
  ax3.plot(x,C_B, color="red", label="Cycle B")
  ax3.legend()